# Importation

In [3]:
from sklearn.metrics import make_scorer,SCORERS,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression,SGDRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras import Sequential, layers
from scipy import stats
import pandas as pd
import numpy as np

# Load dataframe

In [ ]:
data = pd.read_csv("predictions_vinci")
data.head()

,Unnamed: 0,yesterday,prediction,conf_low,conf_high,true,conf_std,Date,perf_pred,perf_true,perf_low,perf_high,garch_pred,vol_variation,prediction_cnn
0,0,101.311661,100.229275,98.853678,101.604872,100.998656,0.701848,2016-09-29,-0.010684,-0.003090,-0.024262,0.002894,21.300782,-0.004677,0.000385
1,1,100.998656,102.686427,101.292483,104.080370,101.535259,0.711209,2016-09-30,0.016711,0.005313,0.002909,0.030512,21.186977,-0.005343,0.000222
2,2,101.535259,101.803834,100.377070,103.230599,101.445808,0.727954,2016-10-03,0.002645,-0.000881,-0.011407,0.016697,21.075146,-0.005278,0.000415
3,3,101.445808,101.608547,100.183373,103.033721,101.997306,0.727143,2016-10-04,0.001604,0.005436,-0.012444,0.015653,21.142841,0.003212,0.000095
4,4,101.997306,101.715455,100.285934,103.144976,100.298096,0.729361,2016-10-05,-0.002763,-0.016659,-0.016779,0.011252,21.256323,0.005367,0.000280


In [ ]:
data.shape

(1131, 15)

In [ ]:
data.loc[:800].tail()

,Unnamed: 0,yesterday,prediction,conf_low,conf_high,true,conf_std,Date,perf_pred,perf_true,perf_low,perf_high,garch_pred,vol_variation,prediction_cnn
796,852,167.608549,167.621782,165.804690,169.438874,166.130379,0.927105,2020-01-30,0.000079,-0.008819,-0.010762,0.010920,91.552026,-0.095035,0.000095
797,853,166.130379,164.764191,162.903386,166.624997,164.570069,0.949408,2020-01-31,-0.008224,-0.009392,-0.019424,0.002977,82.850691,-0.095043,0.000095
798,854,164.570069,164.364885,162.523678,166.206092,165.062804,0.939409,2020-02-03,-0.001247,0.002994,-0.012435,0.009941,70.466986,-0.149470,0.000095
799,855,165.062804,166.784476,164.931914,168.637039,167.854903,0.945202,2020-02-04,0.010430,0.016915,-0.000793,0.021654,91.823870,0.303076,0.000095
800,856,167.854903,167.943377,166.073853,169.812901,170.893370,0.953856,2020-02-05,0.000527,0.018102,-0.010611,0.011665,80.583939,-0.122408,0.000095


In [5]:
feature_to_remove = ["Unnamed: 0","yesterday","prediction","conf_low","conf_high","true","Date"]

In [ ]:
data = data.drop(columns=feature_to_remove).loc[:800]

In [ ]:
data.head()

,conf_std,perf_pred,perf_true,perf_low,perf_high,garch_pred,vol_variation,prediction_cnn
0,0.701848,-0.010684,-0.003090,-0.024262,0.002894,21.300782,-0.004677,0.000385
1,0.711209,0.016711,0.005313,0.002909,0.030512,21.186977,-0.005343,0.000222
2,0.727954,0.002645,-0.000881,-0.011407,0.016697,21.075146,-0.005278,0.000415
3,0.727143,0.001604,0.005436,-0.012444,0.015653,21.142841,0.003212,0.000095
4,0.729361,-0.002763,-0.016659,-0.016779,0.011252,21.256323,0.005367,0.000280


In [ ]:
X = data.copy().drop(columns="perf_true")
y= data["perf_true"]

In [ ]:
mae_arima = mean_absolute_error(data.perf_true,data.perf_pred)
mae_arima

0.00521128782478184

In [ ]:
#to confirme with Arno?
mae_cnn = mean_absolute_error(data.perf_true,data.prediction_cnn)
mae_cnn

0.008093130523205929

# GridSearch SVM

In [ ]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [ ]:

# Instanciate model
model_svm = Pipeline([("scale",StandardScaler()),("svm",SVR())])

# Hyperparameter search space
search_space = {
    'svm__kernel': ["linear", "poly", "rbf", "sigmoid"],
    'svm__C': stats.uniform(0.01, 1000),
    'svm__gamma': stats.loguniform(0.001,10),
    'svm__coef0': stats.uniform(-5,5),
}

# Instanciate Random Search
search_svm = RandomizedSearchCV(
    model_svm, search_space,
    n_jobs=-1, scoring='neg_mean_absolute_error', cv=5, n_iter=1, verbose=0)


search_svm.fit(X,y)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('scale',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             with_std=True)),
                                             ('svm',
                                              SVR(C=1.0, cache_size=200,
                                                  coef0=0.0, degree=3,
                                                  epsilon=0.1, gamma='scale',
                                                  kernel='rbf', max_iter=-1,
                                                  shrinking=True, tol=0.001,
                                                  verbose=False))],
                                      verbose=False),
                   iid='deprecated', n_iter=1, n_jobs=-1,
                

In [ ]:
search_svm.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm',
                 SVR(C=212.1258906498697, cache_size=200,
                     coef0=-3.492154478165015, degree=3, epsilon=0.1,
                     gamma=0.0011984078305023252, kernel='sigmoid', max_iter=-1,
                     shrinking=True, tol=0.001, verbose=False))],
         verbose=False)

In [ ]:
search_svm.best_score_ 

-0.011892484654970713

# GridSearch KNN

In [ ]:
model_knn = Pipeline([("scale",StandardScaler()),("knn",KNeighborsRegressor())])

param_grid_knn =  {'knn__n_neighbors': range(2,50)}
search_knn = GridSearchCV(model_knn, param_grid=param_grid_knn, 
                          cv=5, n_jobs=-1, verbose=0, scoring='neg_mean_absolute_error')
search_knn.fit(X,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('knn',
                                        KNeighborsRegressor(algorithm='auto',
                                                            leaf_size=30,
                                                            metric='minkowski',
                                                            metric_params=None,
                                                            n_jobs=None,
                                                            n_neighbors=5, p=2,
                                                            weights='uniform'))],
                                verbose=False),
             iid='d

In [ ]:
search_knn.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('knn',
                 KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                     metric='minkowski', metric_params=None,
                                     n_jobs=None, n_neighbors=32, p=2,
                                     weights='uniform'))],
         verbose=False)

In [ ]:
search_knn.best_score_

-0.005636257494689957

# GridSearch Ridge

In [ ]:
model_ridge = Pipeline([("scale",StandardScaler()),("ridge",Ridge())])
param_grid_ridge =  {'ridge__alpha': np.linspace(0.0001,2,num=1000)}
search_ridge = GridSearchCV(model_ridge, param_grid=param_grid_ridge, 
                              cv=5, n_jobs=-1, verbose=0, scoring='neg_mean_absolute_error')
search_ridge.fit(X,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('ridge',
                                        Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=False,
                                              random_state=None, solver='auto',
                                              tol=0.001))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'ridge__alpha': array([1.0000...
       1.96196386e+00, 1.96396577e+00, 1.96596767e+00, 1.96796957e+00,
       1.96997147e+00, 1.97197337e+00, 

In [ ]:
search_ridge.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ridge',
                 Ridge(alpha=2.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [ ]:
search_ridge.best_score_

-0.0055310041721148735

# GridSearch Lasso

In [ ]:
model_lasso = Pipeline([("scale",StandardScaler()),("lasso",Lasso())])
param_grid_lasso =  {'lasso__alpha': np.linspace(0.0001,2,num=1000)}
search_lasso = GridSearchCV(model_lasso, param_grid=param_grid_lasso, 
                              cv=5, n_jobs=-1, verbose=0, scoring='neg_mean_absolute_error')
search_lasso.fit(X,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('lasso',
                                        Lasso(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=1000,
                                              normalize=False, positive=False,
                                              precompute=False,
                                              random_state=None,
                                              selection='cyclic', tol=0.0001,
                                              warm_start=False))],
                                verbose=False),
             iid='depreca...
       1.96196386e+00, 1.963965

In [ ]:
search_lasso.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('lasso',
                 Lasso(alpha=0.0001, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False)

In [ ]:
search_lasso.best_score_

-0.005362953921198617

# GridSearch  SGD

In [ ]:
model_sgd = Pipeline([("scale",StandardScaler()),("sgd",SGDRegressor())]) 
param_grid_sgd =  {'sgd__alpha': np.linspace(0.0001,2,num=100),
                  'sgd__penalty': ["l2", "l1", "elasticnet"],
                  "sgd__l1_ratio" :np.linspace(0,1,num=10) }
search_sgd = GridSearchCV(model_sgd, param_grid=param_grid_sgd, 
                              cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
search_sgd.fit(X,y)

Fitting 5 folds for each of 3000 candidates, totalling 15000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 396 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 2332 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done 5580 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done 10108 tasks      | elapsed:   51.4s
[Parallel(n_jobs=-1)]: Done 15000 out of 15000 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('sgd',
                                        SGDRegressor(alpha=0.0001,
                                                     average=False,
                                                     early_stopping=False,
                                                     epsilon=0.1, eta0=0.01,
                                                     fit_intercept=True,
                                                     l1_ratio=0.15,
                                                     learning_rate='invscaling',
                                                     loss='squared_loss',
                               

In [ ]:
search_sgd.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('sgd',
                 SGDRegressor(alpha=0.0001, average=False, early_stopping=False,
                              epsilon=0.1, eta0=0.01, fit_intercept=True,
                              l1_ratio=0.8888888888888888,
                              learning_rate='invscaling', loss='squared_loss',
                              max_iter=1000, n_iter_no_change=5, penalty='l1',
                              power_t=0.25, random_state=None, shuffle=True,
                              tol=0.001, validation_fraction=0.1, verbose=0,
                              warm_start=False))],
         verbose=False)

In [ ]:
search_sgd.best_score_

-0.005286324704918548

# GridSearch Descision Tree

In [ ]:
model_des_tree = Pipeline([("scale",StandardScaler()),("desTree",DecisionTreeRegressor())]) 
param_grid_des_tree =  {'desTree__max_depth': range(2,100),
                  "desTree__min_samples_leaf" :range(2,100) }
search_des_tree = GridSearchCV(model_des_tree, param_grid=param_grid_des_tree, 
                              cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
search_des_tree.fit(X,y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 9604 candidates, totalling 48020 fits


[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 2248 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done 5496 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done 10024 tasks      | elapsed:   52.5s
[Parallel(n_jobs=-1)]: Done 15864 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 22984 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 31416 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 41128 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 48017 out of 48020 | elapsed:  4.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 48020 out of 48020 | elapsed:  4.2min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('desTree',
                                        DecisionTreeRegressor(ccp_alpha=0.0,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features=None,
                                                              max_leaf_nodes=None,
                                                              min_impurity_decrease=0.0,
                                                              min_impurity_split=None,
                                    

In [ ]:
search_des_tree.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('desTree',
                 DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                       max_depth=4, max_features=None,
                                       max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=73, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       presort='deprecated', random_state=None,
                                       splitter='best'))],
         verbose=False)

In [ ]:
search_des_tree.best_score_

-0.005630324226531727

# GridSearch Random forest

In [ ]:
model_rand_tree =  Pipeline([("scale",StandardScaler()),("forest",RandomForestRegressor(criterion="mae"))]) 
param_grid_rand_tree =  {"forest__n_estimators": [10,50, 100, 300, 500, 1000],
                         'forest__max_depth': np.linspace(5,500,10),
                          "forest__min_samples_leaf" :[5] }
search_rand_tree = GridSearchCV(model_rand_tree, param_grid=param_grid_rand_tree, 
                              cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
search_rand_tree.fit(X,y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 17.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('forest',
                                        RandomForestRegressor(bootstrap=True,
                                                              ccp_alpha=0.0,
                                                              criterion='mae',
                                                              max_depth=None,
                                                              max_features='auto',
                                                              max_leaf_nodes=None,
                                                              max_samples=None,
                                                     

In [ ]:
search_rand_tree.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('forest',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mae', max_depth=5.0,
                                       max_features='auto', max_leaf_nodes=None,
                                       max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=5, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=300, n_jobs=None,
                                       oob_score=False, random_state=None,
                                       verbose=0, warm_start=False))],
         verbose=False)

In [ ]:
search_rand_tree.best_score_

-0.005411176637137448

# GridSearch Gradient Boost

In [ ]:
model_gboost = Pipeline([("scale",StandardScaler()),("gboost",GradientBoostingRegressor(criterion="mae"))]) 
param_grid_gboost =  {'gboost__n_estimators': [10,50, 100, 300, 500, 1000],
                      "gboost__learning_rate": [0.0001,0.001,0.003,0.01,0.03,0.1,0.3,1]}
search_gboost = GridSearchCV(model_gboost, param_grid=param_grid_gboost, 
                              cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
search_gboost.fit(X,y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 22.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('gboost',
                                        GradientBoostingRegressor(alpha=0.9,
                                                                  ccp_alpha=0.0,
                                                                  criterion='mae',
                                                                  init=None,
                                                                  learning_rate=0.1,
                                                                  loss='ls',
                                                                  max_depth=3,
                                                    

In [ ]:
search_gboost.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('gboost',
                 GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                           criterion='mae', init=None,
                                           learning_rate=0.01, loss='ls',
                                           max_depth=3, max_features=None,
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators=300,
                                           n_iter_no_change=None,
                               

In [ ]:
search_gboost.best_score_

-0.005492610683150745

# GridSearch Ada Boost

In [ ]:
model_aboost = Pipeline([("scale",StandardScaler()),("aboost",AdaBoostRegressor())])
param_aboost =  {'aboost__n_estimators': [10,50, 100, 300, 500, 1000],
                      "aboost__learning_rate": [0.0001,0.001,0.003,0.01,0.03,0.1,0.3,1],
                      "aboost__loss":['linear', 'square', 'exponential']}
search_aboost = GridSearchCV(model_aboost, param_grid=param_aboost, 
                              cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
search_aboost.fit(X,y)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:  6.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('aboost',
                                        AdaBoostRegressor(base_estimator=None,
                                                          learning_rate=1.0,
                                                          loss='linear',
                                                          n_estimators=50,
                                                          random_state=None))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'aboost__learning_rate': [0.0001, 0.001, 0.003, 0.01,
                                                   0.03, 0

In [38]:
search_aboost.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('aboost',
                 AdaBoostRegressor(base_estimator=None, learning_rate=0.0001,
                                   loss='exponential', n_estimators=50,
                                   random_state=None))],
         verbose=False)

In [39]:
search_aboost.best_score_

-0.005453881741574116

# Stacking 

In [88]:
gboost = search_gboost.best_estimator_
ridge = search_ridge.best_estimator_
lasso = search_lasso.best_estimator_
svm = search_svm.best_estimator_
adaboost = search_aboost.best_estimator_
forest = search_rand_tree.best_estimator_
desTree = search_des_tree.best_estimator_
sgd = search_sgd.best_estimator_
knn = search_knn.best_estimator_




model_stacking = VotingRegressor(
    estimators = [("gboost", gboost),("adaboost", adaboost),("ridge", ridge),("svm", svm),
                 ("lasso", lasso),("forest", forest),("desTree", desTree),("sgd", sgd),("knn", knn)],
    weights = [1,1,1,1,1,1,1,1,1], # to equally weight the two models
    n_jobs=-1
)

score = cross_val_score(model_stacking, X, y, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
print(score.std())
score.mean()

0.0004165790449631416


-0.005484481827457532

# Neural Network

In [6]:
data2 = pd.read_csv("predictions_vinci")
data2 = data2.drop(columns=feature_to_remove)
data2.head()

,conf_std,perf_pred,perf_true,perf_low,perf_high,garch_pred,vol_variation,prediction_cnn
0,0.701848,-0.010684,-0.003090,-0.024262,0.002894,21.300782,-0.004677,0.000385
1,0.711209,0.016711,0.005313,0.002909,0.030512,21.186977,-0.005343,0.000222
2,0.727954,0.002645,-0.000881,-0.011407,0.016697,21.075146,-0.005278,0.000415
3,0.727143,0.001604,0.005436,-0.012444,0.015653,21.142841,0.003212,0.000095
4,0.729361,-0.002763,-0.016659,-0.016779,0.011252,21.256323,0.005367,0.000280


In [8]:
X2 = data2.copy().drop(columns="perf_true")
y2= data2["perf_true"]

In [9]:
X2[800:]

,conf_std,perf_pred,perf_low,perf_high,garch_pred,vol_variation,prediction_cnn
800,0.953856,0.000527,-0.010611,0.011665,80.583939,-0.122408,0.000095
801,1.048799,0.007429,-0.004600,0.019457,91.793703,0.139107,0.000095
802,1.080792,0.004702,-0.007759,0.017164,80.727591,-0.120554,0.000095
803,1.083552,0.010821,-0.001654,0.023296,85.173972,0.055079,0.000095
804,1.082054,0.003450,-0.008919,0.015818,77.207298,-0.093534,0.000095
...,...,...,...,...,...,...,...
1126,1.314958,-0.003736,-0.019600,0.012127,14.462025,0.015707,0.000223
1127,1.191138,0.004214,-0.010282,0.018711,13.084130,-0.068074,0.000427
1128,1.194974,0.003450,-0.010933,0.017832,13.287345,0.015531,0.000235
1129,1.187985,0.000363,-0.013949,0.014675,13.311260,0.001800,0.000345


In [13]:
es = EarlyStopping(patience=10, restore_best_weights=True)
normalizer = Normalization() # Instantiate a "normalizer" layer
normalizer.adapt(X2[:600]) # "Fit" it on the train set


model_nn = Sequential()
model_nn.add(normalizer)
model_nn.add(layers.Dense(100, input_dim=7, activation='tanh'))  
model_nn.add(layers.Dense(50, activation='tanh'))
model_nn.add(layers.Dense(20, activation='tanh'))
model_nn.add(layers.Dense(10, activation='tanh'))
model_nn.add(layers.Dense(20, activation='tanh'))
model_nn.add(layers.Dense(50, activation='tanh'))
model_nn.add(layers.Dense(100, activation='tanh'))
model_nn.add(layers.Dense(1, activation='linear')) 

# STEP 2: OPTIMIZATION METHODS
model_nn.compile(loss='mae', optimizer='adam')

# SETP 3: DATA AND FITTING METHODS
model_nn.fit(X2[:800], y2[:800], batch_size=32, epochs=500, callbacks=[es],verbose=2)
model_nn.evaluate(X2[800:],y2[800:])

Epoch 1/500
25/25 - 1s - loss: 0.0275
Epoch 2/500
25/25 - 0s - loss: 0.0087
Epoch 3/500
25/25 - 0s - loss: 0.0096
Epoch 4/500
25/25 - 0s - loss: 0.0077
Epoch 5/500
25/25 - 0s - loss: 0.0085
Epoch 6/500
25/25 - 0s - loss: 0.0070
Epoch 7/500
25/25 - 0s - loss: 0.0062
Epoch 8/500
25/25 - 0s - loss: 0.0064
Epoch 9/500
25/25 - 0s - loss: 0.0063
Epoch 10/500
25/25 - 0s - loss: 0.0067
Epoch 11/500
25/25 - 0s - loss: 0.0060
Epoch 12/500
25/25 - 0s - loss: 0.0064
Epoch 13/500
25/25 - 0s - loss: 0.0063
Epoch 14/500
25/25 - 0s - loss: 0.0064
Epoch 15/500
25/25 - 0s - loss: 0.0065
Epoch 16/500
25/25 - 0s - loss: 0.0067
Epoch 17/500
25/25 - 0s - loss: 0.0064
Epoch 18/500
25/25 - 0s - loss: 0.0066
Epoch 19/500
25/25 - 0s - loss: 0.0063
Epoch 20/500
25/25 - 0s - loss: 0.0059
Epoch 21/500
25/25 - 0s - loss: 0.0066
Epoch 22/500
25/25 - 0s - loss: 0.0063
Epoch 23/500
25/25 - 0s - loss: 0.0058
Epoch 24/500
25/25 - 0s - loss: 0.0060
Epoch 25/500
25/25 - 0s - loss: 0.0060
Epoch 26/500
25/25 - 0s - loss: 0.

0.015291730873286724

In [60]:
mae_arima2 = mean_absolute_error(data2.perf_true,data2.perf_pred)
mae_arima2

0.007416762485498858

In [61]:
mae_cnn2 = mean_absolute_error(data2.perf_true,data2.prediction_cnn)
mae_cnn2

0.011540322337057218

# GridSearch SVM 2

In [ ]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [14]:

# Instanciate model
model_svm2 = Pipeline([("scale",StandardScaler()),("svm",SVR())])

# Hyperparameter search space
search_space = {
    'svm__kernel': ["linear", "poly", "rbf", "sigmoid"],
    'svm__C': stats.uniform(0.01, 1000),
    'svm__gamma': stats.loguniform(0.001,10),
    'svm__coef0': stats.uniform(-5,5),
}

# Instanciate Random Search
search_svm2 = RandomizedSearchCV(
    model_svm2, search_space,
    n_jobs=-1, scoring='neg_mean_absolute_error', cv=5, n_iter=1, verbose=0)


search_svm2.fit(X2,y2)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('scale',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             with_std=True)),
                                             ('svm',
                                              SVR(C=1.0, cache_size=200,
                                                  coef0=0.0, degree=3,
                                                  epsilon=0.1, gamma='scale',
                                                  kernel='rbf', max_iter=-1,
                                                  shrinking=True, tol=0.001,
                                                  verbose=False))],
                                      verbose=False),
                   iid='deprecated', n_iter=1, n_jobs=-1,
                

In [15]:
search_svm2.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm',
                 SVR(C=839.8558013361466, cache_size=200,
                     coef0=-4.22039902528053, degree=3, epsilon=0.1,
                     gamma=0.3158924805723352, kernel='sigmoid', max_iter=-1,
                     shrinking=True, tol=0.001, verbose=False))],
         verbose=False)

In [16]:
search_svm2.best_score_

-5757.935330599809

# GridSearch KNN 2

In [17]:
model_knn2 = Pipeline([("scale",StandardScaler()),("knn",KNeighborsRegressor())])

param_grid_knn =  {'knn__n_neighbors': range(2,50)}
search_knn2 = GridSearchCV(model_knn2, param_grid=param_grid_knn, 
                          cv=5, n_jobs=-1, verbose=0, scoring='neg_mean_absolute_error')
search_knn2.fit(X2,y2)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('knn',
                                        KNeighborsRegressor(algorithm='auto',
                                                            leaf_size=30,
                                                            metric='minkowski',
                                                            metric_params=None,
                                                            n_jobs=None,
                                                            n_neighbors=5, p=2,
                                                            weights='uniform'))],
                                verbose=False),
             iid='d

In [18]:
search_knn2.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('knn',
                 KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                     metric='minkowski', metric_params=None,
                                     n_jobs=None, n_neighbors=34, p=2,
                                     weights='uniform'))],
         verbose=False)

In [19]:
search_knn2.best_score_

-0.008223683937343882

# GridSearch Ridge 2



In [20]:
model_ridge2 = Pipeline([("scale",StandardScaler()),("ridge",Ridge())])
param_grid_ridge =  {'ridge__alpha': np.linspace(0.0001,2,num=1000)}
search_ridge2 = GridSearchCV(model_ridge2, param_grid=param_grid_ridge, 
                              cv=5, n_jobs=-1, verbose=0, scoring='neg_mean_absolute_error')
search_ridge2.fit(X2,y2)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('ridge',
                                        Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=False,
                                              random_state=None, solver='auto',
                                              tol=0.001))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'ridge__alpha': array([1.0000...
       1.96196386e+00, 1.96396577e+00, 1.96596767e+00, 1.96796957e+00,
       1.96997147e+00, 1.97197337e+00, 

In [21]:
search_ridge2.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ridge',
                 Ridge(alpha=2.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [22]:
search_ridge2.best_score_

-0.007843342859763914

# GridSearch Lasso 2

In [24]:
model_lasso2 = Pipeline([("scale",StandardScaler()),("lasso",Lasso())])
param_grid_lasso =  {'lasso__alpha': np.linspace(0.0001,2,num=1000)}
search_lasso2 = GridSearchCV(model_lasso2, param_grid=param_grid_lasso, 
                              cv=5, n_jobs=-1, verbose=0, scoring='neg_mean_absolute_error')
search_lasso2.fit(X2,y2)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('lasso',
                                        Lasso(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=1000,
                                              normalize=False, positive=False,
                                              precompute=False,
                                              random_state=None,
                                              selection='cyclic', tol=0.0001,
                                              warm_start=False))],
                                verbose=False),
             iid='depreca...
       1.96196386e+00, 1.963965

In [25]:
search_lasso2.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('lasso',
                 Lasso(alpha=0.002101901901901902, copy_X=True,
                       fit_intercept=True, max_iter=1000, normalize=False,
                       positive=False, precompute=False, random_state=None,
                       selection='cyclic', tol=0.0001, warm_start=False))],
         verbose=False)

In [26]:
search_lasso2.best_score_

-0.0076426701158435984

# GridSearch  SGD

In [28]:
model_sgd2 = Pipeline([("scale",StandardScaler()),("sgd",SGDRegressor())]) 
param_grid_sgd =  {'sgd__alpha': np.linspace(0.0001,2,num=100),
                  'sgd__penalty': ["l2", "l1", "elasticnet"],
                  "sgd__l1_ratio" :np.linspace(0,1,num=10) }
search_sgd2 = GridSearchCV(model_sgd2, param_grid=param_grid_sgd, 
                              cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
search_sgd2.fit(X2,y2)

Fitting 5 folds for each of 3000 candidates, totalling 15000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 396 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 2332 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 5580 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done 9912 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 15000 out of 15000 | elapsed:  2.1min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('sgd',
                                        SGDRegressor(alpha=0.0001,
                                                     average=False,
                                                     early_stopping=False,
                                                     epsilon=0.1, eta0=0.01,
                                                     fit_intercept=True,
                                                     l1_ratio=0.15,
                                                     learning_rate='invscaling',
                                                     loss='squared_loss',
                               

In [29]:
search_sgd2.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('sgd',
                 SGDRegressor(alpha=0.0001, average=False, early_stopping=False,
                              epsilon=0.1, eta0=0.01, fit_intercept=True,
                              l1_ratio=0.6666666666666666,
                              learning_rate='invscaling', loss='squared_loss',
                              max_iter=1000, n_iter_no_change=5, penalty='l1',
                              power_t=0.25, random_state=None, shuffle=True,
                              tol=0.001, validation_fraction=0.1, verbose=0,
                              warm_start=False))],
         verbose=False)

In [30]:
search_sgd2.best_score_

-0.0075275868387744945

# GridSearch Descision Tree

In [31]:
model_des_tree2 = Pipeline([("scale",StandardScaler()),("desTree",DecisionTreeRegressor())]) 
param_grid_des_tree =  {'desTree__max_depth': range(2,100),
                  "desTree__min_samples_leaf" :range(2,100) }
search_des_tree2 = GridSearchCV(model_des_tree2, param_grid=param_grid_des_tree, 
                              cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
search_des_tree2.fit(X2,y2)

Fitting 5 folds for each of 9604 candidates, totalling 48020 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 396 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 2332 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 5580 tasks      | elapsed:   32.2s
[Parallel(n_jobs=-1)]: Done 10108 tasks      | elapsed:   59.3s
[Parallel(n_jobs=-1)]: Done 15948 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 23068 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 31500 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 41212 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 48020 out of 48020 | elapsed:  4.8min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('desTree',
                                        DecisionTreeRegressor(ccp_alpha=0.0,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features=None,
                                                              max_leaf_nodes=None,
                                                              min_impurity_decrease=0.0,
                                                              min_impurity_split=None,
                                    

In [32]:
search_des_tree2.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('desTree',
                 DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                       max_depth=4, max_features=None,
                                       max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=40, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       presort='deprecated', random_state=None,
                                       splitter='best'))],
         verbose=False)

In [33]:
search_des_tree2.best_score_

-0.008148874061844256

# GridSearch Random forest

In [34]:
model_rand_tree2 =  Pipeline([("scale",StandardScaler()),("forest",RandomForestRegressor(criterion="mae"))]) 
param_grid_rand_tree =  {"forest__n_estimators": [10,50, 100, 300, 500, 1000],
                         'forest__max_depth': np.linspace(5,500,10),
                          "forest__min_samples_leaf" :[5] }
search_rand_tree2 = GridSearchCV(model_rand_tree2, param_grid=param_grid_rand_tree, 
                              cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
search_rand_tree2.fit(X2,y2)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 14.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 30.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('forest',
                                        RandomForestRegressor(bootstrap=True,
                                                              ccp_alpha=0.0,
                                                              criterion='mae',
                                                              max_depth=None,
                                                              max_features='auto',
                                                              max_leaf_nodes=None,
                                                              max_samples=None,
                                                     

In [35]:
search_rand_tree2.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('forest',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mae', max_depth=5.0,
                                       max_features='auto', max_leaf_nodes=None,
                                       max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=5, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=1000, n_jobs=None,
                                       oob_score=False, random_state=None,
                                       verbose=0, warm_start=False))],
         verbose=False)

In [36]:
search_rand_tree2.best_score_

-0.00781822709851894

# GridSearch Gradient Boost

In [37]:
model_gboost2 = Pipeline([("scale",StandardScaler()),("gboost",GradientBoostingRegressor(criterion="mae"))]) 
param_grid_gboost =  {'gboost__n_estimators': [10,50, 100, 300, 500, 1000],
                      "gboost__learning_rate": [0.0001,0.001,0.003,0.01,0.03,0.1,0.3,1]}
search_gboost2 = GridSearchCV(model_gboost2, param_grid=param_grid_gboost, 
                              cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
search_gboost2.fit(X2,y2)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 23.5min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 40.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('gboost',
                                        GradientBoostingRegressor(alpha=0.9,
                                                                  ccp_alpha=0.0,
                                                                  criterion='mae',
                                                                  init=None,
                                                                  learning_rate=0.1,
                                                                  loss='ls',
                                                                  max_depth=3,
                                                    

In [38]:
search_gboost2.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('gboost',
                 GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                           criterion='mae', init=None,
                                           learning_rate=0.01, loss='ls',
                                           max_depth=3, max_features=None,
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators=300,
                                           n_iter_no_change=None,
                               

In [39]:
search_gboost2.best_score_

-0.007838722551611527

# GridSearch Ada Boost

In [41]:
model_aboost2 = Pipeline([("scale",StandardScaler()),("aboost",AdaBoostRegressor())])
param_aboost =  {'aboost__n_estimators': [10,50, 100, 300, 500, 1000],
                      "aboost__learning_rate": [0.0001,0.001,0.003,0.01,0.03,0.1,0.3,1],
                      "aboost__loss":['linear', 'square', 'exponential']}
search_aboost2 = GridSearchCV(model_aboost2, param_grid=param_aboost, 
                              cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
search_aboost2.fit(X2,y2)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:  7.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('aboost',
                                        AdaBoostRegressor(base_estimator=None,
                                                          learning_rate=1.0,
                                                          loss='linear',
                                                          n_estimators=50,
                                                          random_state=None))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'aboost__learning_rate': [0.0001, 0.001, 0.003, 0.01,
                                                   0.03, 0

In [42]:
search_aboost2.best_estimator_

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('aboost',
                 AdaBoostRegressor(base_estimator=None, learning_rate=0.003,
                                   loss='exponential', n_estimators=100,
                                   random_state=None))],
         verbose=False)

In [43]:
search_aboost2.best_score_

-0.00807476465048657

# Stacking 

In [ ]:
gboost2 = search_gboost2.best_estimator_
ridge2 = search_ridge2.best_estimator_
lasso2 = search_lasso2.best_estimator_
svm2 = search_svm2.best_estimator_
adaboost2= search_aboost2.best_estimator_
forest2 = search_rand_tree2.best_estimator_
desTree2 = search_des_tree2.best_estimator_
sgd2 = search_sgd2.best_estimator_
knn2 = search_knn2.best_estimator_




model_stacking2 = VotingRegressor(
    estimators = [("gboost", gboost2),("adaboost", adaboost2),("ridge", ridge2),("svm", svm2),
                 ("lasso", lasso2),("forest", forest2),("desTree", desTree2),("sgd", sgd2),("knn", knn2)],
    weights = [1,1,1,1,1,1,1,1,1], # to equally weight the two models
    n_jobs=-1
)

score = cross_val_score(model_stacking2, X2, y2, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
print(score.std())
score.mean()

0.0004165790449631416


-0.005484481827457532